# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../output_data/cities_new.csv")
weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Hobart,40,AU,1616357830,94,-42.8794,147.3294,57.20,5.75
1,Mataura,100,NZ,1616357943,95,-46.1927,168.8643,54.00,3.00
2,Saint-Augustin,100,CA,1616357944,97,51.2260,-58.6502,33.26,13.00
3,Rikitea,0,PF,1616357944,71,-23.1203,-134.9692,80.13,7.54
4,Abalak,3,RU,1616357944,94,58.1286,68.5944,12.36,3.40


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps
gmaps.configure(api_key=g_key)

locations = weather_df[["Lat","Lng"]]
humidity = weather_df["Humidity"].astype(float)
#print(locations)
#print(humidity)

In [9]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

A Jupyter Widget

In [10]:
max_intensity

100.0

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
#Conditions
##A max temperature lower than 80 degrees but higher than 70.
##Wind speed less than 10 mph.
##Zero cloudiness.
##Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

narrow_df = weather_df.loc[(weather_df["Wind Speed"] <=10)&(weather_df["Cloudiness"]==0)&
                           (weather_df["Max Temp"]<=80)&(weather_df["Max Temp"]>=70)]
narrow_df.head()



,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
10,Calvinia,0,ZA,1616357945,23,-31.4707,19.7760,71.31,4.68
96,Karratha,0,AU,1616357993,58,-20.7377,116.8463,78.30,5.17
101,Kharan,0,PK,1616357994,38,28.5833,65.4167,70.07,8.19
115,Roebourne,0,AU,1616357997,59,-20.7833,117.1333,78.31,5.88
128,Kānker,0,IN,1616358001,34,20.2719,81.4931,73.76,3.56


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
# make hotel_df
hotel_df = narrow_df.loc[:,["City","Country","Lat","Lng"]]
hotel_df = hotel_df.reset_index()
# add column hotel name
hotel_df["Hotel Name"] = ""
hotel_df.head()

,index,City,Country,Lat,Lng,Hotel Name
0,10,Calvinia,ZA,-31.4707,19.7760,
1,96,Karratha,AU,-20.7377,116.8463,
2,101,Kharan,PK,28.5833,65.4167,
3,115,Roebourne,AU,-20.7833,117.1333,
4,128,Kānker,IN,20.2719,81.4931,


In [14]:

# Use the lat/lng we recovered to identify hotels

for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "types": "lodging",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        #print(hotel_data)
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

,index,City,Country,Lat,Lng,Hotel Name
0,10,Calvinia,ZA,-31.4707,19.7760,Die Lemoenhuis
1,96,Karratha,AU,-20.7377,116.8463,ibis Styles Karratha
2,101,Kharan,PK,28.5833,65.4167,Dil Aram Bangla
3,115,Roebourne,AU,-20.7833,117.1333,Harding River Caravan Park
4,128,Kānker,IN,20.2719,81.4931,STARiHOTEL Kanker
5,138,Jati,PK,24.3539,68.2671,Cafe Lala Rehan Restaurant
6,178,Tura,IN,25.5198,90.2201,Hotel RIKMAN Continental
7,180,Olavarría,AR,-36.8927,-60.3225,Hotel Santa Rosa
8,186,Safaga,EG,26.7292,33.9365,Lamar Resort Abu Soma
9,243,Bosaso,SO,11.2842,49.1816,Red Sea Hotel


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig

A Jupyter Widget